<a href="https://colab.research.google.com/github/bcazaubon72/Coursera_Capstone/blob/master/List_of_postal_codes_of_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing of requests 

In [0]:

pip install requests

 *Importing of Beautifulsoup,requests and pandas libraries*

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


*Loading url into a beautifulsoup object.italicized text*

In [0]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content,'lxml')

Find table element in beautiful object  and reading in html object.

In [0]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

 *Removing all  rows marked 'Not assigned' in column 'Borough'*

In [0]:

data =df[df['Borough']!= 'Not assigned']

*Group dataframe by 'Postal Code' so 'Neighborhood' with similar postal code can be placed on one line*

In [0]:
data1 = data.groupby(df['Postal Code'],as_index=False)

*Turn  dataframe groupby object into dateframe. Then reset index and drop the index column* 

In [0]:
data2=data1.head(179)
data2=data2.reset_index().drop(columns='index')

In [8]:
data2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
data2.shape

(103, 3)

In [30]:
sr= data2['Postal Code'].sort_values(ascending=True)
sr[0]

'M3A'

In [10]:
pip install geocoder

     |████████████████████████████████| 102kB 2.4MB/s 


In [0]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(data2['Postal Code']))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

*Reading in the Geospatial_Coordinates csv and merging the newly formed dataframe with the previous built one*

In [0]:
data3 = pd.read_csv('/content/Geospatial_Coordinates.csv')


In [60]:
new_df = pd.merge(data2,data3)
new_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
